In [2]:
import plotly.express as px
import geopandas as gpd
import pandas as pd
from idMunicipios import obter_malha
from shapely.geometry import shape
from typing import Dict, List

def obter_malhas_para_municipios(codigos_municipio: List[str]) -> List[Dict]:
    malhas = []
    for codigo in codigos_municipio:
        malha = obter_malha(codigo)
        malhas.append(malha)
    return malhas

def criar_gdf_malhas(malhas: List[Dict]) -> gpd.GeoDataFrame:
    features = [feature for malha in malhas if malha for feature in malha.get('features', [])]
    geometries = [shape(feature['geometry']) for feature in features]
    properties = [feature['properties'] for feature in features]

    gdf = gpd.GeoDataFrame(properties, geometry=geometries, crs='epsg:4326')
    return gdf

df_municipios = pd.read_csv('parahybaCities.csv', engine='python')

df_municipios.rename(columns={'Código IBGE': 'CD_MUN'}, inplace=True)
df_municipios['CD_MUN'] = df_municipios['CD_MUN'].astype(str)

codigos_municipio = df_municipios['CD_MUN'].tolist()
malhas = obter_malhas_para_municipios(codigos_municipio)
gdf = criar_gdf_malhas(malhas)


df_municipios = df_municipios.drop(columns=['Tecnologia', 'Tipo de compromisso', 'Compromisso', 
                                             'Origem compromisso', 'Operadora com compromisso', 
                                             'Status compromisso', 'Prazo compromisso', 
                                             'Operadora'], axis=1)

df_municipios = df_municipios.drop_duplicates()

df_municipios['% moradores cobertos'] = df_municipios['% moradores cobertos'].str.replace('%', '').str.replace(',', '.').str.strip()
df_municipios['% moradores cobertos'] = pd.to_numeric(df_municipios['% moradores cobertos'], errors='coerce')
df_municipios['% moradores cobertos'] = df_municipios['% moradores cobertos'].fillna(0)

print("Colunas após renomear:", df_municipios.columns)

gdf['CD_MUN'] = df_municipios['CD_MUN'].astype(int)
gdf = gdf.merge(df_municipios, on='CD_MUN', how='left')

print("Colunas após merge:", gdf.columns)
print(gdf.head())

grafico = px.choropleth_mapbox(
    gdf,
    geojson=gdf.geometry.__geo_interface__,  
    locations=gdf.index,  
    featureidkey="properties.CD_MUN",  
    color="% moradores cobertos", 
    mapbox_style="carto-positron",
    zoom=6,
    center={"lat": -7.239, "lon": -35.881},
    opacity=0.5,
    color_continuous_scale="Viridis"
)

grafico.update_layout(margin={"t": 0, "l": 0, "r": 0, "b": 0})
grafico.show()


KeyboardInterrupt: 